In [1]:
import sys

print(sys.version)

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


#  디스크립터란

- 클래스의 속성을 보호하기 위해 만드는 특별한 클래스이다.
- 디스크립터로 만드는 속성을 보호 속성으로 만든다.
- 속성에 대한 접근을 이름으로 조회하고 갱신할 수 있다.

- 기본 프로토콜 규약에서는 3가지가 있다.

> `__get__`, `__set__`, `__delete__` 중 하나가 구현되어 있는 클래스를 디스크립터 클래스

## 1.  디스크립터를 만들고 내부 인스턴스에 속성 넣기

## 디스크립터 클래스는 정의

- 디스크립터 객체를 생성할 때 값을 관리하는 필드와 이 속성을 처리하는 3개의 메소드를 정의한다.
- 값 저장은 일단 디스크립터 객체에 만들어서 보관하는 것부터 알아본다.


In [94]:
 def aaa(x) :
    print(x)
    return "aaa"

In [95]:
aaa.__get__(aaa)()

<function aaa at 0x000002A4876F5430>


'aaa'

In [96]:
aaa(6)

6


'aaa'

In [99]:
aaa

<function __main__.aaa(x)>

In [3]:
dir(aaa)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [37]:
class Descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("get")
        return self.value       # 디스크립터 객체의 값을 조회
    
    def __set__(self, instance, value):
        print("set")
        self.value = value      # 디스크립터 객체의 값을 갱신
        
    def __delete__(self, instance):
        print("delete")
        del self.value

## 반드시 클래스 속성으로 정의 

- 디스크립터 객체는 반드시 클래스 속성으로 정의한다. 


In [50]:
class A:
    value = Descriptor(10) # instance
    value1 = Descriptor(10) # instance
    
    def __init__(self, value, value1, value2):
        self.value = value
        self.value1 = value1
        self.value2 = value2

In [51]:
a = A(30, 40, 999)

set
set


In [54]:
a.value, a.value1, a.value2

get
get


(30, 40, 999)

In [55]:
a.__dict__

{'value2': 999}

{'value': 10}

### 객체를 생성한다.


In [3]:
a = A()

### 디스크립터 객체에 값을 보관해서 클래스 A의 객체에는 아무런 속성도 없다.

In [4]:
a.__dict__

{}

### 값을 조회하기

- 디스크립터는 클래스 내부의 속성을 확인하고 이를 이용해서 내부의 메소드를 호출해서 처리
- 클래스 A의 네임스페이스를 조회하면 두 개의 디스크립터 속성이 만들어져 있다.

In [5]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x7fa5792dae10>,
              'value1': <__main__.Descriptor at 0x7fa5792da490>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

### 속성의 값을 조회하기

In [6]:
type(a).__dict__['value']

In [7]:
type(a).__dict__['value'].__get__

<bound method Descriptor.__get__ of <__main__.Descriptor object at 0x7fa5792dae10>>

In [8]:
type(a).__dict__['value'].__get__(a, type(a))

get


10

In [9]:
a.value     # 디스크립터 클래스의 __get__ 메소드를 조회한다. 

get


10

###  디스크립터는 클래스의 속성을 읽어서 갱신도 함

In [10]:
type(a).__dict__['value'].__set__(a, 12)

set


In [11]:
a.value

get


12

In [12]:
a.value1 = 13  

set


In [13]:
a.value1

get


13

###  실제 삭제된 거을 디스크리터로 만들어진 객체 

In [14]:
type(a).__dict__['value'].__delete__(a)

delete


In [15]:
type(a).__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x7fa5792dae10>,
              'value1': <__main__.Descriptor at 0x7fa5792da490>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [16]:
try : 
    a.value
except Exception as e :
    print(e)

get
'Descriptor' object has no attribute 'value'


In [17]:
del a.value1 

delete


In [18]:
type(a).__dict__

mappingproxy({'__module__': '__main__',
              'value': <__main__.Descriptor at 0x7fa5792dae10>,
              'value1': <__main__.Descriptor at 0x7fa5792da490>,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [19]:
try : 
    a.value1
except Exception as e :
    print(e)

get
'Descriptor' object has no attribute 'value'


## 다시 세팅하면 디스크립터가 작동되어 처리됨 

In [20]:
a.value =123

set


In [21]:
a.value

get


123

In [22]:
type(a).__dict__['value1'].__set__(a,999)

set


In [23]:
a.value1

get


999

# 2.  디스크립터 프로토콜이 없다면..

- 속성을 관리하는 클래스를 정의하고 메인 클래스의 속성에 객체를 생성한다.
- 메인 클래스 속성 즉 객체에 속성을 추가한다.
- 이 속성에 정보를 조회한다.
- 점연산자를 연속적으로 사용해야 한다.
- 디스크립터 클래스를 만들고 처리하는 것보다 더 복잡한 구조를 가진다.

### 일반 클래스를 정의한다. 

In [24]:
class D:
    def __getattribute__(self, name):
        print("getattribute")
        return super().__getattribute__(name)


### 클래스 내부의 속성에 객체를 할당한다.

In [25]:
class A:
    value = D()

### 객체 내부의 value 속성은 객체이미므로 이 객체의 속성 data를 추가한다.

In [26]:
a = A()
a.value.data = 1

### 객체 내부의 data 값을 조회한다. 

In [27]:
a.value.data

getattribute


1

## 3.  읽기 전용  `__get__`  만 가진 디스크립터

- 메인 클래스의 속성을 읽기 전용으로만 만들 수 있다.

In [119]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [131]:
class FirstDescriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, owner):
        print("get")
        return self.value
    
    def __set__(self, instance, value):
        print("set")
        self.value = value
    

class A:
    a = FirstDescriptor(3)
    
    def __init__(self,a):
        setattr(self, 'a' , a)

In [129]:
a = A(9999)
a.a

set
get


9999

In [130]:
a.__dict__

{}

### 디스크립터 읽기 전용 만들기

- `__set__` 을 정의했지만 예외를 발생시켜서 읽기 전용으로 만들 수 있다.  



In [138]:
class Descriptor:
    def __init__(self, value):
        self.value = value
        
    def __get__(self, instance, onwer):
        print("get")
        return self.value
    
#     def __set__(self, instance, value):
#         print("set")
#         self.value = value  # 쓰기 가능
        
    def __set__(self, instance, value):
        # 읽기전용
        print("set")
        raise AttributeError("read only")
        


In [139]:
class A:
    a = Descriptor(3)

In [140]:
a = A()

In [141]:
a.a

get


3

In [142]:
try : 
    a.a = 12
except Exception as e :
    print(e)

set
read only


# 4. 디스크립터에서 메인 클래스의 객체로 저장하는 위치

- 실제 처리할 때는 메인클래스의 객체 내부의 속성으로 값을 처리한다.
- 메인 클래스의 인스턴스를 받아서 저장할 위치를 인스턴스로 변경한다.

### 디스크립터에 메인 클래스의 객체에 저장 처리를 하지 않았다

In [162]:
class descriptor:
    def __get__(self, instance, owner):
        print('get', owner)
        return instance.value
    
    def __set__(self, instance, value):
        print('set')
        instance.value = value


In [163]:
### 메인클래스 속성에 디스크립터를 할당하지만 실제 객체와 동일한 변수가 없다.

In [164]:
class A:
    x = descriptor()
    y = descriptor()

In [165]:
a = A()

In [166]:
a.__dict__

{}

In [167]:
a.x = 123

set


In [168]:
a.y = 999

set


In [169]:
a.__dict__

{'value': 999}

In [170]:
try :
    print(a.x)
except Exception as e :
    print(e)

get <class '__main__.A'>
999


### 인스턴스 객체 내에 속성 저장하고 처리하기

In [191]:
class descriptor1:
    
    def __init__(self, name):
        self.name = "_" + name 
        
    def __get__(self, instance, owner):
        print('get', instance)
        return instance.__dict__.get(self.name,0) 
    
    def __set__(self, instance, value):
        print('set')
        instance.__dict__[self.name] = value



In [199]:
class AA:
    x = descriptor1('x')
    y = descriptor1('y')

In [200]:
aa = AA()

In [201]:
aa.x

get <__main__.AA object at 0x000002A4869DA700>


0

In [202]:
aa.x = 100; aa.y = 999

set
set


In [203]:
aa.__dict__

{'_x': 100, '_y': 999}

In [204]:
aa.x, aa.y

get <__main__.AA object at 0x000002A4869DA700>
get <__main__.AA object at 0x000002A4869DA700>


(100, 999)

### 메인 클래스 속성을 가져오기

In [305]:
class descriptor2:
    
    def __init__(self, type_) :
        self.type_ = type_
    
    def __set_name__(self, owner, name ):
        self.name = "_"+ name
        
    def __get__(self, instance, owner):
        print('get', instance)
        return instance.__dict__.get(self.name,0) 
    
    def __set__(self, instance, value):
        print('set')
        if isinstance(value, self.type_):
            instance.__dict__[self.name] = value
        else:
            raise TypeError("타입이 안 맞습니다.")

In [266]:
class AAA:
    x = descriptor2(int)
    y = descriptor2(float)
    
    def __init__(self, x,y) :
        self.x = x
        self.y = y
        
    def add(self):
        return self.x + self.y
    
    def sub(self):
        return self.x - self.y
    
    def truediv(self):
        return self.x / self.y
    
    def floordiv(self):
        return self.x // self.y
    
    def mul(self):
        return self.x * self.y

In [267]:
class AAA_:
    
    def __init__(self, x,y) :
        self.x = x
        self.y = y
        
    def add(self):
        return self.x + self.y
    
    def sub(self):
        return self.x - self.y
    
    def truediv(self):
        return self.x / self.y
    
    def floordiv(self):
        return self.x // self.y
    
    def mul(self):
        return self.x * self.y

In [268]:
aaa_ = AAA_(10, 20)

In [270]:
aaa_.__dict__

{'x': 10, 'y': 20}

In [249]:
aaa = AAA(10, 20.0)

set
set


In [250]:
aaa.x

get <__main__.AAA object at 0x000002A4869DAA90>


10

In [251]:
aaa.add(), aaa.sub(), aaa.truediv(), aaa.floordiv(), aaa.mul()

get <__main__.AAA object at 0x000002A4869DAA90>
get <__main__.AAA object at 0x000002A4869DAA90>
get <__main__.AAA object at 0x000002A4869DAA90>
get <__main__.AAA object at 0x000002A4869DAA90>
get <__main__.AAA object at 0x000002A4869DAA90>
get <__main__.AAA object at 0x000002A4869DAA90>
get <__main__.AAA object at 0x000002A4869DAA90>
get <__main__.AAA object at 0x000002A4869DAA90>
get <__main__.AAA object at 0x000002A4869DAA90>
get <__main__.AAA object at 0x000002A4869DAA90>


(30.0, -10.0, 0.5, 0.0, 200.0)

In [231]:
aaa.__dict__

{'_x': 10, '_y': 20}

In [256]:
class BBB(AAA) :
    z = descriptor2(int)
    def __init__(self,x,y,z) :
        super().__init__(x,y)
        self.z = z
    
    def add(self):
        return self.x + self.y + self. z

In [257]:
bbb = BBB(10, 20.0,10)

set
set
set


In [258]:
bbb.add()

get <__main__.BBB object at 0x000002A4868F7580>
get <__main__.BBB object at 0x000002A4868F7580>
get <__main__.BBB object at 0x000002A4868F7580>


40.0

In [254]:
aaa = AAA(10,20.)

set
set


In [214]:
aaa.x = 100

set


In [215]:
aaa.x

get <__main__.AAA object at 0x000002A4868F7A60>


100

In [11]:
BBB.__dict__

mappingproxy({'__module__': '__main__',
              'z': <__main__.descriptor2 at 0x7fad02c7bd10>,
              '__init__': <function __main__.BBB.__init__(self, x, y, z)>,
              '__doc__': None})

In [8]:
bbb = BBB(10,20,30)

set
set
set


In [10]:
bbb.x, bbb.y, bbb.z

get <__main__.BBB object at 0x7fad02c7b690>
get <__main__.BBB object at 0x7fad02c7b690>
get <__main__.BBB object at 0x7fad02c7b690>


(10, 20, 30)

## 5. 조회할 때 속성을 초기화하기

In [55]:
class descriptor3:
    
    def __set_name__(self, owner, name ):
        self.name = "_"+ name
        
    def __get__(self, instance, owner):
        print('get', instance)
        return instance.__dict__.setdefault(self.name,0) 
    
    def __set__(self, instance, value):
        print('set')
        instance.__dict__[self.name] = value

In [56]:
class AAAA :
    x = descriptor3()

In [57]:
aaaa = AAAA()

In [58]:
aaaa.x

get <__main__.AAAA object at 0x7fa57933c150>


0

In [59]:
aaaa.__dict__

{'_x': 0}

In [60]:
aaaa.x =100

set


In [61]:
aaaa.__dict__

{'_x': 100}

In [62]:
aaaa.x

get <__main__.AAAA object at 0x7fa57933c150>


100

## 6.  함수를 저장해서 처리하는 디스크립터 만들기 

- 디스크립터 클래스를 만들고 데코레이터를 처리할 수 있다.
- 이름으로 접근하면 내부으 함수가 실행된다.


### 디스크립터 조회할 때 저장된 함수를 반환한다

In [306]:
class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print("get")
        return self.func(instance) # 메소드는 첫번째 인자로 self 자기자신을 받아야되기 때문에

### 디스크립터로 메소드를 데코레이터 처리한다.

In [307]:
class A:
    a = descriptor2(int)
    b = descriptor2(int)
    c = descriptor2(int)
    def __init__(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
    
    @descriptor
    def func(self):
        return self.a + self.b + self.c


In [308]:
a = A(1, 2, 3)

set
set
set


In [309]:
a.func # __call__ X

get
get <__main__.A object at 0x000002A4869DA370>
get <__main__.A object at 0x000002A4869DA370>
get <__main__.A object at 0x000002A4869DA370>


6

### 함수의 인자를 받도록 수정

- 부분 함수 처리를 위해 pattial 로 처리한다. 
- 이름으로 조회하면 부분함수가 반환되고 메소드의 인자를 추가적으로 넣어서 처리할 수 있다

In [286]:
from functools import partial

In [287]:
help(partial)

Help on class partial in module functools:

class partial(builtins.object)
 |  partial(func, *args, **keywords) - new function with partial application
 |  of the given arguments and keywords.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __reduce__(...)
 |      Helper for pickle.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __setattr__(self, name, value, /)
 |      Implement setattr(self, name, value).
 |  
 |  __setstate__(...)
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  ----------------------------------------------------------------------

In [293]:
import functools

In [294]:
for i in dir(functools) :
    print(i) if not i.startswith("_") else _

RLock
WRAPPER_ASSIGNMENTS
WRAPPER_UPDATES
cached_property
cmp_to_key
get_cache_token
lru_cache
namedtuple
partial
partialmethod
recursive_repr
reduce
singledispatch
singledispatchmethod
total_ordering
update_wrapper
wraps


In [295]:
def outer(func):
    def inner(*args, **kwargs) :
        return func(*args, **kwargs)
    return inner

In [296]:
@outer
def add(x,y):
    return x + y

In [289]:
add = partial(add, 10)

In [290]:
add(20)

30

In [297]:
from functools import partial


class descriptor:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        print('get')
        return partial(self.func, instance)

In [298]:
class A:
    @descriptor
    def func(self, a, b, c):
        return a + b + c

In [299]:
a = A()

In [302]:
a.func(1,2,3)

get


6

## 7. 도전 : 디스크립터로 구현

### 인스턴스를 생성할때 반지름을 입력받는 원 객체 / 원의 속성을 가지고 있는..

> 반지름은 mutable    
> 그 외에 지름, 둘레, 넓이는 immutable

### 함수를 저장하는 방식으로

### 갱신가능한 디스크립터 정의

In [310]:
class MutableAttribute:
    def __init__(self, value=None):
        self.value = value
        
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        self.value = value
        
    def __delete__(self, instance):
        del self.value



### 갱신 불가능한 디스크립터 정의

In [311]:
class ImmutableAttribute:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, owner):
        return self.func(instance)
    
    def __set__(self, instance, value):
        raise AttributeError("read only")
        
    def __delete__(self, instance):
        raise AttributeError("read only")

### 메인 클래스 정의하고 메소드를 변경불가능하도록 처리

In [348]:
class Circle:
    pi = 3.1415
    radius = MutableAttribute(10)
    diameter = ImmutableAttribute(lambda self : self.radius * 2)
    
#     @ImmutableAttribute
#     def diameter(self):
#         return self.radius * 2
    
    @ImmutableAttribute
    def circumference(self):
        return self.radius * self.pi * 2
    
    @ImmutableAttribute
    def area(self):
        return self.radius **2 * 2

In [349]:
c = Circle()

In [350]:
c.radius

10

In [351]:
c.diameter

20

In [352]:
c.area

200

In [353]:
c.radius = 100

In [354]:
c.area

20000

In [355]:
try :
    c.area = 123
except Exception as e :
    print(e)

read only


In [356]:
(lambda x,y : x + y)(10, 20)

30

외부 람다 함수의 매개변수로 함수를 넣고 내부 람다 함수를 반환시키는 걸 변수에 저장하고 데코레이터로 사용

In [388]:
aaa = lambda func : (lambda *args, **kwargs : func( *args, **kwargs))

In [389]:
@aaa
def add(x,y):
    return x + y

In [390]:
add(10, 20)

30

In [391]:
def sub(x,y):
    return x - y

In [392]:
sub = aaa(sub)

In [393]:
sub(10, 5)

5

구분함수로 만들려면 x 를 먼저 넣어준다

In [394]:
aaa = lambda func, x : (lambda *args, **kwargs : func(x, *args, **kwargs))

In [395]:
def sub(x,y):
    return x - y

In [396]:
sub = aaa(sub, 10)

In [397]:
sub(15)

-5

구분함수라서 데코레이터는 안된다

In [398]:
@aaa
def add(x,y):
    return x + y

TypeError: <lambda>() missing 1 required positional argument: 'x'

In [399]:
import types

In [402]:
isinstance((lambda x:x),types.LambdaType)

True

In [403]:
isinstance((lambda x:x),types.FunctionType)

True